This function computes closed null geodesics of $ S(\mathbf{x}_0, t)-\mu \mathbf{I} $, where $ S(\mathbf{x}_0, t) = \begin{pmatrix} S^{11} && S^{12} \\ S^{12} && S^{22} \end{pmatrix} $ is the (symmetric) rate of strain tensor.

| Name | Type (Shape) | Description |
| --- | --- | --- |
| X | array (Ny, Nx) | X-meshgrid|
| Y | array (Ny, Nx) | Y-meshgrid|
| mu | float | $ \mu $|
| interp_phi_prime | interpolant | $ \dot{\phi} $|
| d_threshold | float | threshold distance between start of closed null-geodesic and maximum allowed first return distance to starting point |
| S11 | array(Ny, Nx) | array containing the S11-element of rate of strain tensor|
| iteration_length | float | length of dummy time interval used for computation of elliptic LCSs |
| n_iterations | int | resolution of dummy time interval |
| solODE_closed_curve | list | list containing the closed solution curves ($ \mathbf{x}_0, \phi $) |
| x0mu | list | list containing x-coordinates of initial conditions $ x_0(\mu) $ |
| y0mu | list | list containing y-coordinates of initial conditions $ y_0(\mu) $ |

In [1]:
import sys, os

# get current directory
path = os.getcwd()

# get parent directory
parent_directory = os.path.sep.join(path.split(os.path.sep)[:-3])

# add utils folder to current working path
sys.path.append(parent_directory+"/subfunctions/utils")

In [2]:
# Import numpy
import numpy as np

# function computing initial conditions (depending on \mu)
from ipynb.fs.defs.init_level_set import _init_level_set

# find closed curve
from ipynb.fs.defs.closed_curve import closed_curve

# RK4 integrator
from ipynb.fs.defs.RK4_integration import RK4_integration

In [ ]:
def closed_null_geodesics(X, Y, mu, interp_phi_prime, d_threshold, S11, iteration_length = 10, n_iterations = 6000, verbose = False):
    '''
    Compute closed null geodesics. The algorithm returns closed null geodesics coinciding with elliptic OECS.
    
    Parameters:
        X:                      array (Ny, Nx),  X-meshgrid.
        Y:                      array (Ny, Nx),  Y-meshgrid.
        mu:                     float, mu --> stretching used to compute ICs of null geodesics.
        interp_phi_prime:       Interpolant for phi_prime.
        d_threshold:            float, threshold value for re-intersection of closed curve
        S11:                    array (Ny, Nx), array of S_{11}.
        iteration_length:       float, length of dummy time interval used for computation of elliptic OECSs
        n_iterations:           int, resolution of dummy time interval
        verbose:                bool, if True, function reports progress at every 100th iteration.          
        
    Returns:
        solODEcurves:          list, solutions of ODE = set of closed null geodesics.
        x0mu:                  list, x-coordinates of ICs.
        y0mu:                  list, y-coordinates of ICs.
    '''
    
    # domain where the rate of strain field is defined
    defined_domain = np.isfinite(S11).astype(int)
    
    # compute initial conditions
    x0mu, y0mu, phi0mu = _init_level_set(X, Y, S11, mu)
    
    # define integration domain of dummy variable.
    # Note if you do not find any structures, then I recommend increasing the value "iteration_length". 
    # For the majority of examples the algorithm converges much sooner. 
    # An upper bound on the dummy time-interval is included in order to avoid long computation times.
    s = [0, iteration_length]
    
    # define resolution of trajectories
    # This resolution can also be refined.
    s_eval = np.linspace(s[0], s[1], n_iterations)
    
    ds = s_eval[1]-s_eval[0]
    
    # number of initial conditions
    len_x0mu = len(x0mu)
    
    solODE = np.zeros((len(s_eval), 3, len_x0mu))
    
    # initial conditions
    solODE[0, :, :] = np.array([x0mu, y0mu, phi0mu])
    
    for i in range(len(s_eval)-1):
        
        solODE[i+1, :, :] = RK4_integration(solODE[i, :, :], ds, interp_phi_prime)
    
    solODE_closed_curves = []
    
    print("Computation of closed null geodesics: Started...")
    
    # iterate over all initial conditions [x0mu, y0mu, phi0mu]
    for j in range(len_x0mu):
    
        
        # store x, y, phi
        # we do not solve the ODE for all ICs as this would require long computation times.
        # for higher accuracy just replace the subsequente three lines of codes with the commented version
        x = solODE[::2,0, j] #solODE[:,0,j]
        y = solODE[::2,1, j] #solODE[:,1,j]
        phi = solODE[::2,2, j] #solODE[:,2,j]
    
        if verbose:
            if j%100==0:
                print("Percentage completed: ", np.around(np.around(j/len_x0mu, 4)*100,2))
    
        # Check if curve is closed after completing one full cycle and find curve with minimum re-intersection distance.
        x_closed, y_closed, phi_closed = closed_curve(x, y, phi, d_threshold, X, Y, defined_domain)
        
        # store solutions
        solODE_closed_curves.append([x_closed, y_closed, phi_closed])
    
    print("Computation of closed null geodesics: Done!")
    
    return solODE_closed_curves, [x0mu, y0mu]